In [18]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import MatchingAlgorithm 

import importlib 
importlib.reload(MatchingAlgorithm)


<module 'MatchingAlgorithm' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\MatchingAlgorithm.py'>

In [16]:
import FeatureEngineering
importlib.reload(FeatureEngineering)


<module 'FeatureEngineering' from 'c:\\Users\\XieLo\\OneDrive\\Documents\\NUS\\NUS_ACADEMICS\\YEAR 4\\Sem 2\\BT4103 Capstone\\Github\\Capstone-Project\\FeatureEngineering.py'>

### Running algorithm

In [3]:
# INPUT YOUR API KEY

API_key = ''

In [4]:
orders_df = pd.read_csv("Simulated Data/order_data_1576.csv")
catchments_df = pd.read_csv("Simulated Data/catchment_data_1576.csv")
phlebs_df = pd.read_csv("Simulated Data/phleb_data_1576.csv")

In [5]:
#orders_df = orders_df.iloc[:15]
#phlebs_df = phlebs_df.iloc[:2]
catchments_df = catchments_df.iloc[:1] #remove this for multi-ends 

In [69]:
coordinate_list = FeatureEngineering.get_coordinates_list(orders_df, catchments_df, phlebs_df)
time_matrix =  FeatureEngineering.create_time_matrix(coordinate_list, API_key)

In [6]:
time_matrix = np.loadtxt('timeMatrix.txt')

In [129]:
output = MatchingAlgorithm.run_algorithm_version_timeMatrix(orders_df, catchments_df, phlebs_df, time_matrix)
print(output)

{
  "Metadata": {
    "Locations": [
      {
        "Location Index": 0,
        "Coordinate": "28.453012121886808,77.04885293025431",
        "Order Id": "Ending Location"
      },
      {
        "Location Index": 1,
        "Coordinate": "28.446275918004297,77.05666911676877",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 2,
        "Coordinate": "28.436374190228392,77.10818368839077",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 3,
        "Coordinate": "28.46043105269203,77.12523221919234",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 4,
        "Coordinate": "28.44673371496172,77.08386447523183",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 5,
        "Coordinate": "28.450267421182843,77.0411748655223",
        "Order Id": "Starting Location"
      },
      {
        "Location Index": 6,
        "Coordinate": "28.448895764608586,7

### Prescriptive Analytics

In [7]:

phlebs_df['service_rating'] = 5 #ensure all service rating is the same

In [38]:
demand_ratios = [(1, 0, 0), (0.8, 0.2, 0), (0.6, 0.2, 0.2),  (0.4, 0.4, 0.2), (0.2, 0.6, 0.2), (0, 0.6, 0.4), (0, 0.8, 0.2), (0, 1, 0),
                 (0, 0.5, 0.5), (0, 0.4, 0.6), (0, 0.2, 0.8), (0, 0, 1)]

full_ratios = []
full_numbers = []
full_costs = []
full_times = []

for demand in demand_ratios:
    list_of_ratios, list_of_total_transit_time, list_of_total_cost, list_of_numbers, list_of_jsons, list_of_chosen_phlebs = MatchingAlgorithm.run_prescriptive_analysis(
        orders_df, catchments_df, phlebs_df, time_matrix, demand[0], demand[1], demand[2])
    full_ratios.append(list_of_ratios)
    full_numbers.append(list_of_numbers)
    full_costs.append(list_of_total_cost)
    full_times.append(list_of_total_transit_time)




In [39]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

mean_ratios = [np.mean(list_of_ratios, axis=0) for list_of_ratios in full_ratios]

fig = make_subplots(rows=3, cols=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[0] for ratio in mean_ratios],
    name="Regular" 
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[1] for ratio in mean_ratios],
    name="Premium" 
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[2] for ratio in mean_ratios],
    name="Special" 
), row=3, col=1)


fig.update_layout(height=800, width=1300, title_text="Demand Ratios over Supply Ratios")
fig.show()

In [54]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

mean_numbers = [np.mean(list_of_total_cost, axis=0) for list_of_total_cost in full_numbers]

fig = make_subplots(rows=1, cols=1)

fig.append_trace(go.Scatter(
    x=[str(i) for i in demand_ratios], 
    y=[ratio[0] + ratio[1] + ratio[2] for ratio in mean_numbers]
), row=1, col=1)



fig.update_layout(height=800, width=1300, title_text="Demand Number over Supply Ratios")
fig.show()

### Visualisation

In [50]:
# import pygeos
import RouteVisualisation
import geopandas as gpd

importlib.reload(RouteVisualisation)

gpd.options.use_pygeos = False

In [51]:
addresses_list = FeatureEngineering.get_coordinates_list(orders_df, catchments_df, phlebs_df)

In [53]:
json_result = json.loads(list_of_jsons[1])

polygon = gpd.read_file("Simulation\Gurugram_sample_Polygon.geojson")
addresses_list = addresses_list

RouteVisualisation.visualise_routes(json_result, polygon)

c:\Users\XieLo\anaconda3\envs\env_dlib\lib\site-packages\osmnx\utils_geo.py:335: ShapelyDeprecationWarning:

Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.

c:\Users\XieLo\anaconda3\envs\env_dlib\lib\site-packages\osmnx\utils_geo.py:426: ShapelyDeprecationWarning:

Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.

c:\Users\XieLo\anaconda3\envs\env_dlib\lib\site-packages\osmnx\utils_geo.py:426: ShapelyDeprecationWarning:

Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.



Creating route travelled by Phlebotomist ID #0
[1, 0]
Coords: 28.446275918004297,77.05666911676877
Node:  2229150123 268158667
Route:  [2229150123, 2229150119, 2229150135, 2229150140, 582870143, 4223719387, 582870142, 4223749840, 582870152, 2229150494, 582870155, 1469167085, 1469167105, 6094519323, 266381173, 266381187, 267127781, 266381206, 266381188, 268158668, 2008921242, 268158667]
Drawing Phlebotomist ID #0's route...
Map created
Creating route travelled by Phlebotomist ID #1
[2, 0]
Coords: 28.436374190228392,77.10818368839077
Node:  1811153392 268158667
Route:  [1811153392, 1811153397, 1811153460, 4146508835, 1811153446, 1811153414, 1811153411, 4166108732, 9867830439, 4220923024, 4220922996, 3953762561, 3953762598, 3953762642, 3953762643, 3953762669, 3953762672, 3953762673, 1868729070, 1868729077, 4130098960, 4130099332, 267085330, 271335425, 271335530, 9865451046, 5487689965, 1648219221, 4757946170, 4757946054, 5228569746, 537508795, 5228569757, 6322678288, 1358732526, 135873252

NetworkXNoPath: No path to 1564008434.